<a href="https://colab.research.google.com/github/Joseph-dh-woods/IGDB-CSV-generator/blob/main/IGDB_API_user_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Run this cell first to install the IGDB API

In [ ]:
pip install igdb-api-v4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Run this cell next to set up some required stuff

In [ ]:
#@title Imports and Function Definitions
import requests
from igdb.wrapper import IGDBWrapper
import json
import time
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd
import re
import datetime
from IPython.display import clear_output 
def makeBar(perc):
  barStart = '|\x1b[42m'
  barEnd = '|'
  loadEnd = '\x1b[0m'
  barFill = ''
  barMid = ''
  for _ in range(perc):
    barMid += ' '
  barWhole = barStart+barMid+loadEnd+barEnd
  while (len(barWhole) < 32):
    barFill += ' '
    barWhole = barStart+barMid+loadEnd+barFill+barEnd
  barWhole = barWhole + '\n' +barWhole
  return barWhole
def progressBar(num, den):
  perc = num/den
  perc = perc*100
  perc = round(perc/5)
  print(makeBar(perc))
  print(str((den-num)*5) + " Seconds left")
def unicodeReplace(found):
  found = found.replace('\\u0026', '&')
  found = found.replace('\\u0027', "'")
  found = found.replace('\\u2022', "•")
  found = found.replace('•', '-')
  found = found.replace('\"', '"')
  found = found.replace('\n', ' ')
  return found
def getID(inGame):
  found = re.search('"id": (.+?),', inGame).group(1)
  return found
def getName(inGame):
  found = re.search('"name": "(.+?)"', inGame).group(1)
  found = unicodeReplace(found)
  return found
def getCollection(inGame):
  found = re.search('"collection": (.+?),', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('collections', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"name": "(.+?)"', myJson).group(1)
    found = found + plainTxt + ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getRelease(inGame):
  found = re.search('"first_release_date": (.+?),', inGame).group(1)
  timestamp = datetime.datetime.fromtimestamp(int(found))
  return timestamp.strftime('%Y-%m-%d')
def getGenres(inGame):
  found = re.search('"genres": \[(.+?)\]', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('genres', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"name": "(.+?)"', myJson).group(1)
    found = found + plainTxt + ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getThemes(inGame):
  found = re.search('"themes": \[(.+?)\]', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('themes', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"name": "(.+?)"', myJson).group(1)
    found = found + plainTxt + ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getCompanies(inGame):
  found = re.search('"involved_companies": \[(.+?)\]', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('involved_companies', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"company": (.+?),', myJson).group(1)
    byteArray2 = wrapper.api_request('companies', 'fields *; where id = ' + str(plainTxt) + ';')
    nameByte = byteArray2.decode('utf8').replace("'", '"')
    plainName = re.search('"name": "(.+?)"', nameByte).group(1)
    found = found + plainName
    if '"developer": true,' in myJson:
      found += "(DEV)"
    if '"publisher": true,' in myJson:
      found += "(PUB)"
    if '"porting": true,' in myJson:
      found += "(PORT)"
    if '"supporting": true,' in myJson:
      found += "(SPRT)" 
    found += ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getSummary(inGame):
  found = re.search('summary": "(.+?)",', inGame).group(1)
  found = unicodeReplace(found)
  return found
def getFranchises(inGame):
  found = re.search('franchises": \[(.+?)\]', inGame).group(1)
  found = found.replace(',', '')
  idList = found.split()
  found = ''
  for numin in idList:
    byteArray = wrapper.api_request('franchises', 'fields *; where id = ' + str(numin) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    myJson = myJson.replace('\n', ' ')
    myJson = ' '.join(myJson.split())
    plainTxt = re.search('"name": "(.+?)",', myJson).group(1)
    found = found + plainTxt + ', '
  found = found.rstrip()
  found = found.strip(',')
  found = unicodeReplace(found)
  return found
def getCover(inGame):
  byteArray = wrapper.api_request('games', 'fields cover; where id = ' + str(inGame) + ';')
  myJson = byteArray.decode('utf8').replace("'", '"')
  if "cover" in myJson:
    plaintext = re.search('"cover": (.+?)\n', myJson).group(1)
    byteArray = wrapper.api_request('covers', 'fields *; where id = ' + str(plaintext) + ';')
    myJson = byteArray.decode('utf8').replace("'", '"')
    plaintext = re.search('"image_id": "(.+?)"', myJson).group(1)
    found = "https://images.igdb.com/igdb/image/upload/t_cover_big/" + plaintext + ".png"
    return found
  else:
    return "no Cover found"
def makeItem(jason):
  temp = ['','','','','','','','','','']
  game = jason.replace('\n', ' ')
  game = ' '.join(jason.split())
  if '"id"' in game:
    temp[0] = getID(game)
  if '"name"' in game:
    temp[1] = getName(game)
    print("adding " + temp[1])
  if '"collection"' in game:
    temp[2] = getCollection(game)
  if '"first_release_date"' in game:
    temp[3] = getRelease(game)
  if '"genres"' in game:
    temp[4] = getGenres(game)
    time.sleep(.01)
  if "themes" in game:
    temp[5] = getThemes(game)
    time.sleep(.01)
  if '"involved_companies"' in game:
    temp[6] = getCompanies(game)
    time.sleep(.01)
  if '"summary"' in game:
    temp[7]= getSummary(game)
  if '"franchises"' in game:
    temp[8] = getFranchises(game)
    time.sleep(.01)
  if '"cover"' in game:
    temp[9] = getCover(temp[0])
    time.sleep(.01)
  return temp

Enter YOUR id and secret gotten from the Twitch developer page.

If you don't know what this means, or don't have these go here 

https://api-docs.igdb.com/#about 

and follow the create account sections

!!THIS SECTION MUST BE RUN EVERYTIME YOU CREATE A NEW LIST

In [ ]:
#@title Authorization
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
worksheet = gc.open('ID_INPUT').sheet1
IDlist = []
addedGames = []
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
for x in rows:
  IDlist.append(x[0])
  addedGames.append(x[1])
clientID = "p4drjdnuirwnzc9i1rs74fv8sdhtbi" #@param {type:"string"}
clientSecret = "anz0yqx9ppk7k649a1j80qclr9n1zv" #@param {type:"string"}
requestURL = "https://id.twitch.tv/oauth2/token?client_id=" + clientID + "&client_secret=" + clientSecret + "&grant_type=client_credentials"
getToken = requests.post(requestURL)
s = getToken.text
sI = s.split("\"")
accessToken = sI[3]
accessID = clientID
wrapper = IGDBWrapper(accessID, accessToken)

Search by title and select which games you want to add from a list using the number on the left (you can include multiple games each search just type in all the numbers you want to add at once, so to add 1, 2, and 5, type 125)

This section is optional, if you filled the A column of ID_INPUT with IDs yourself just skip this

In [ ]:
#@title Add ID directly
ID = 117019 #@param {type:"number"}
clear_output()
print("Added games So Far:")
print(addedGames)
byteArray = wrapper.api_request('games', 'fields name; where id = ' + str(ID) + ';')
myJson = byteArray.decode('utf8').replace("'", '"')
if(myJson != '[]'):
  results = myJson.split('{')
  results.pop(0)
  resultsList = []
  for _ in results:
    tmp = ['', '']
    tmp[0] = getID(_)
    tmp[1] = getName(_)
    resultsList.append(tmp)

  for _ in resultsList:
    print("Added : " + str(_[1]) + '(' + str(_[0]) + ')')
    IDlist.append(_[0])
    addedGames.append(_[1])

  rangeCall = 'A1' + ':A' + str(len(IDlist))
  cell_list = worksheet.range(rangeCall)
  numin = 0
  for cell in cell_list:
    cell.value = IDlist[numin]
    numin += 1
  worksheet.update_cells(cell_list)

  rangeCall = 'B1' + ':B' + str(len(addedGames))
  cell_list = worksheet.range(rangeCall)
  numin = 0
  for cell in cell_list:
    cell.value = addedGames[numin]
    numin += 1
  worksheet.update_cells(cell_list)
else:
  print("No game found")


Added games So Far:
[]
Added : Again(117019)


In [ ]:
#@title Add IDs to ID_INPUT by searching
showCover = "Don't include Covers (Faster)" #@param ["Include Covers (Slower)", "Don't include Covers (Faster)"]
cont = 1
while cont != 0:
  clear_output()
  print("Added games So Far:")
  print(addedGames)
  titleIN = input("(enter -1 to finish)\nTitle: ")
  if titleIN == '-1':
    cont = 0
    break
  byteArray = wrapper.api_request('games', 'fields name; limit 9; search "' + str(titleIN) + '";')
  myJson = byteArray.decode('utf8').replace("'", '"')
  results = myJson.split('{')
  results.pop(0)
  resultsList = []
  for _ in results:
    tmp = ['', '']
    tmp[0] = getID(_)
    tmp[1] = getName(_)
    resultsList.append(tmp)

  for _ in resultsList:
    print(str(resultsList.index(_)+1) + ': ' + str(_[1]) + '(' + str(_[0]) + ')')
    if showCover == "Include Covers (Slower)":
      print(getCover(_[0]))
  addlist = (input("games to add:"))
  addlist = (set(addlist))
  cleaned = [ x for x in addlist if x.isdigit() ]
  addlist = cleaned

  if ('0' in addlist):
    continue
  for _ in addlist:
    IDlist.append(resultsList[int(_)-1][0])
    addedGames.append(resultsList[int(_)-1][1])

rangeCall = 'A1' + ':A' + str(len(IDlist))
cell_list = worksheet.range(rangeCall)
numin = 0
for cell in cell_list:
  cell.value = IDlist[numin]
  numin += 1
worksheet.update_cells(cell_list)

rangeCall = 'B1' + ':B' + str(len(addedGames))
cell_list = worksheet.range(rangeCall)
numin = 0
for cell in cell_list:
  cell.value = addedGames[numin]
  numin += 1
worksheet.update_cells(cell_list)

Added games So Far:
['Again', 'Ace Attorney Investigations: Miles Edgeworth', 'Apollo Justice: Ace Attorney', 'Brain Age 2: More Training in Minutes a Day!', 'Brain Age: Train Your Brain in Minutes a Day!', 'Elite Beat Agents', 'Fossil Fighters', 'Guitar Hero: On Tour', 'Hotel Dusk: Room 215', 'Kingdom Hearts 358/2 Days', 'Kingdom Hearts Re:coded', 'Lego Rock Band', 'New Super Mario Bros.', 'Nintendogs: Dachshund & Friends', 'Phoenix Wright: Ace Attorney − Trials and Tribulations', 'Phoenix Wright: Ace Attorney − Justice for All', 'Phoenix Wright: Ace Attorney', 'Pokémon Black', 'Pokémon Black Version 2', 'Pokémon Conquest', 'Pokémon Diamond', 'Pokémon HeartGold', 'Pokémon Mystery Dungeon: Blue Rescue Team', 'Pokémon Platinum', 'Professor Layton and the Diabolical Box', 'Professor Layton and the Curious Village', 'Professor Layton and the Unwound Future', 'Professor Layton and the Last Specter', 'Rhythm Heaven', 'Rock Band 3', 'Spectrobes', 'Spectrobes: Beyond the Portals', 'Super Mari

{'spreadsheetId': '1EIuO_yd6lYaU8NeJnEugpHNmKPi3C_lOmzBy76JIiMU',
 'updatedCells': 42,
 'updatedColumns': 1,
 'updatedRange': 'Sheet1!B1:B42',
 'updatedRows': 42}

When ID_INPUT is filled with the IGDB IDs of the games you want (make sure they are all in column A) run this to fill in the rest of the game information, this takes a while so just let it run, and then go download ID_input, or copy the data to another file, just remember to clear ID_Input when you're done for the next time you want to use this program

In [ ]:
#@title Populate ID_INPUT with game information
# Make the API call to populate list games from the IDs, then send them to function makeItem for formatting from JSON to csv
games = []

# refreshes getid and makes sure they are int
IDlist = []
rows = worksheet.get_all_values()
for x in rows:
  IDlist.append(int(x[0]))
cntr = 1
for ID in IDlist:
  rq = 'fields name, age_ratings, first_release_date, genres, involved_companies, summary, themes, url, collection, franchises, cover; where id = ' + str(ID) + ';'
  byteArray = wrapper.api_request(
              'games',
              rq
            )
  myJson = byteArray.decode('utf8').replace("'", '"')
  myJson = unicodeReplace(myJson)
  clear_output()
  progressBar(cntr, len(IDlist))
  cntr+=1
  games.append(makeItem(myJson))
  time.sleep(.1)

#Stores the games in the original sheets file with headers
rangeCall = 'A1:I1'
cell_list = worksheet.range(rangeCall)
headerRow = ["ID", "Title", "Series", "Initial Release", "Genres", "Themes", "Companies", "Summary", "Franchise"]
numin = 0
for cell in cell_list:
  cell.value = headerRow[numin]
  numin +=1
worksheet.update_cells(cell_list)
clear_output()
print("Creating sheets file")
print(makeBar(21))
for _ in games:
  rangeCall = 'A' + str(games.index(_)+2) + ':J' + str(games.index(_)+2)
  cell_list = worksheet.range(rangeCall)
  numin = 0
  for cell in cell_list:
    cell.value = _[numin]
    numin += 1
  worksheet.update_cells(cell_list)
print("Finished")


Creating sheets file
|                     |
|                     |
Finished


From here you can open ID_INPUT and insert a row at the beggining, and use the formula =IMAGE(_cell_) with cell being the cover url cell add covers to your google sheet